In [20]:
pip install transformers datasets scikit-learn torch kaggle


In [21]:
!pip install tensorflow tensorflow-text keras

In [22]:

import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow_text as text


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:


# Change this path to match where you uploaded the file in your Drive
file_path = '/content/drive/MyDrive/Trichy cllg sem 2 data/IVA Labs/7.News_Category_Dataset_v3.json'

# Load the dataset
df = pd.read_json(file_path, lines=True)
df


,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [25]:
df = df[['headline', 'category']].dropna()
df

,headline,category
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
3,The Funniest Tweets From Parents This Week (Se...,PARENTING
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS
...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS


In [26]:
df.category.value_counts()

,count
category,
POLITICS,35602
WELLNESS,17945
ENTERTAINMENT,17362
TRAVEL,9900
STYLE & BEAUTY,9814
PARENTING,8791
HEALTHY LIVING,6694
QUEER VOICES,6347
FOOD & DRINK,6340


In [27]:
# Clean the headlines
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z']", " ", text)
    return text.lower().strip()

df['headline'] = df['headline'].apply(clean_text)
df

,headline,category
0,over million americans roll up sleeves for o...,U.S. NEWS
1,american airlines flyer charged banned for li...,U.S. NEWS
2,of the funniest tweets about cats and dogs thi...,COMEDY
3,the funniest tweets from parents this week sept,PARENTING
4,woman who called cops on black bird watcher lo...,U.S. NEWS
...,...,...
209522,rim ceo thorsten heins' 'significant' plans fo...,TECH
209523,maria sharapova stunned by victoria azarenka i...,SPORTS
209524,giants over patriots jets over colts among m...,SPORTS
209525,aldon smith arrested ers linebacker busted ...,SPORTS


In [28]:
label_enc = LabelEncoder()
df['category_enc'] = label_enc.fit_transform(df['category'])
df

,headline,category,category_enc
0,over million americans roll up sleeves for o...,U.S. NEWS,35
1,american airlines flyer charged banned for li...,U.S. NEWS,35
2,of the funniest tweets about cats and dogs thi...,COMEDY,5
3,the funniest tweets from parents this week sept,PARENTING,22
4,woman who called cops on black bird watcher lo...,U.S. NEWS,35
...,...,...,...
209522,rim ceo thorsten heins' 'significant' plans fo...,TECH,32
209523,maria sharapova stunned by victoria azarenka i...,SPORTS,28
209524,giants over patriots jets over colts among m...,SPORTS,28
209525,aldon smith arrested ers linebacker busted ...,SPORTS,28


In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    df['headline'], df['category_enc'], test_size=0.2, random_state=42, stratify=df['category_enc']
)

In [30]:
#  Tokenization
MAX_LEN = 20
VOCAB_SIZE = 10000

vectorizer = layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_LEN)
vectorizer.adapt(X_train.values)

X_train_vec = vectorizer(X_train)
X_test_vec = vectorizer(X_test)

In [31]:
# positional Encoding
def positional_encoding(length, depth):
    depth = depth/2
    positions = np.arange(length)[:, np.newaxis]
    depths = np.arange(depth)[np.newaxis, :]/depth

    angle_rates = 1 / (10000**depths)
    angle_rads = positions * angle_rates

    pos_encoding = np.concatenate([np.sin(angle_rads), np.cos(angle_rads)], axis=-1)
    return tf.cast(pos_encoding, dtype=tf.float32)

In [32]:
#  Scaled Dot-Product Attention
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super().__init__()
        assert embed_dim % num_heads == 0

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads

        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)
        return output

In [33]:
#  Transformer block
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))

In [34]:
# Token + Positional Embedding
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [36]:


# Build model
embed_dim = 64
num_heads = 4
ff_dim = 128
num_classes = df['category_enc'].nunique()

inputs = layers.Input(shape=(MAX_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x, training=True)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])





In [37]:
history = model.fit(X_train_vec, y_train, batch_size=64, epochs=5, validation_split=0.1)

Epoch 1/5
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 107s 41ms/step - accuracy: 0.3636 - loss: 2.5517 - val_accuracy: 0.5491 - val_loss: 1.6859
Epoch 2/5
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 140s 41ms/step - accuracy: 0.5649 - loss: 1.6290 - val_accuracy: 0.5532 - val_loss: 1.6366
Epoch 3/5
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 92s 39ms/step - accuracy: 0.5929 - loss: 1.4762 - val_accuracy: 0.5605 - val_loss: 1.6161
Epoch 4/5
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 143s 39ms/step - accuracy: 0.6155 - loss: 1.3720 - val_accuracy: 0.5638 - val_loss: 1.6101
Epoch 5/5
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 102s 43ms/step - accuracy: 0.6371 - loss: 1.2823 - val_accuracy: 0.5620 - val_loss: 1.6075


In [38]:
model.evaluate(X_test_vec, y_test)

1310/1310 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - accuracy: 0.5647 - loss: 1.6140


[1.6111133098602295, 0.5636186003684998]

In [39]:
def predict_category(texts, model, vectorizer, label_encoder):
    cleaned = [clean_text(t) for t in texts]
    vec = vectorizer(tf.constant(cleaned))
    preds = model.predict(vec)
    predicted_indices = np.argmax(preds, axis=1)
    return label_encoder.inverse_transform(predicted_indices)

In [40]:
sample_headlines = [
    "NASA discovers new planet that could support life",
    "Trump speaks at election rally in Florida",
    "New iPhone released with improved camera",
    "Tips for saving money on a tight budget",
    "Why yoga is the ultimate stress reliever"
]

predicted_labels = predict_category(sample_headlines, model, vectorizer, label_enc)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 575ms/step


In [41]:

for headline, label in zip(sample_headlines, predicted_labels):
    print(f" Headline: {headline}\n ----> Predicted Category: {label}\n")


 Headline: NASA discovers new planet that could support life
 ----> Predicted Category: SCIENCE

 Headline: Trump speaks at election rally in Florida
 ----> Predicted Category: POLITICS

 Headline: New iPhone released with improved camera
 ----> Predicted Category: TECH

 Headline: Tips for saving money on a tight budget
 ----> Predicted Category: MONEY

 Headline: Why yoga is the ultimate stress reliever
 ----> Predicted Category: WELLNESS



In [57]:
# 3. Load data
file_path = '/content/drive/MyDrive/Trichy cllg sem 2 data/IVA Labs/7.News_Category_Dataset_v3.json'
df = pd.read_json(file_path, lines=True)
df = df[['headline', 'category']].dropna()

In [58]:

# 4. Clean text
def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"[^a-zA-Z']", " ", text)
    return text.lower().strip()

df['headline'] = df['headline'].apply(clean_text)
df

,headline,category
0,over million americans roll up sleeves for o...,U.S. NEWS
1,american airlines flyer charged banned for li...,U.S. NEWS
2,of the funniest tweets about cats and dogs thi...,COMEDY
3,the funniest tweets from parents this week sept,PARENTING
4,woman who called cops on black bird watcher lo...,U.S. NEWS
...,...,...
209522,rim ceo thorsten heins' 'significant' plans fo...,TECH
209523,maria sharapova stunned by victoria azarenka i...,SPORTS
209524,giants over patriots jets over colts among m...,SPORTS
209525,aldon smith arrested ers linebacker busted ...,SPORTS


In [59]:
# 5. Label encoding
label_enc = LabelEncoder()
df['category_enc'] = label_enc.fit_transform(df['category'])
df

,headline,category,category_enc
0,over million americans roll up sleeves for o...,U.S. NEWS,35
1,american airlines flyer charged banned for li...,U.S. NEWS,35
2,of the funniest tweets about cats and dogs thi...,COMEDY,5
3,the funniest tweets from parents this week sept,PARENTING,22
4,woman who called cops on black bird watcher lo...,U.S. NEWS,35
...,...,...,...
209522,rim ceo thorsten heins' 'significant' plans fo...,TECH,32
209523,maria sharapova stunned by victoria azarenka i...,SPORTS,28
209524,giants over patriots jets over colts among m...,SPORTS,28
209525,aldon smith arrested ers linebacker busted ...,SPORTS,28


In [60]:

# 6. Split
X_train, X_test, y_train, y_test = train_test_split(
    df['headline'], df['category_enc'], test_size=0.2, random_state=42, stratify=df['category_enc']
)

In [61]:
# 7. Tokenizer
MAX_LEN = 20
VOCAB_SIZE = 20000

vectorizer = layers.TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_LEN)
vectorizer.adapt(X_train.values)

X_train_vec = vectorizer(X_train)
X_test_vec = vectorizer(X_test)

In [62]:
# 8. Transformer pieces
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        positions = tf.range(start=0, limit=tf.shape(x)[-1], delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        return tf.matmul(weights, value), weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)
        attention, _ = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat = tf.reshape(attention, (batch_size, -1, self.num_heads * self.projection_dim))
        return self.combine_heads(concat)

In [63]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs)
        out1 = self.layernorm1(inputs + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output, training=training))


In [64]:

# 9. Build model
embed_dim = 128
num_heads = 4
ff_dim = 256
num_classes = df['category_enc'].nunique()

inputs = layers.Input(shape=(MAX_LEN,))
embedding_layer = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, embed_dim)
x = embedding_layer(inputs)
x = layers.LayerNormalization()(x)  # Added normalization here
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x, training=True)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding_4       │ (None, 20, 128)             │       2,562,560 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_9                │ (None, 20, 128)             │             256 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block_4                  │ (None, 20, 128)             │         132,480 │
│ (TransformerBlock)                   │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_2           │ (None, 128)                 │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_34 (Dense)                     │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_35 (Dense)                     │ (None, 42)                  │           5,418 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,717,226 (10.37 MB)

 Trainable params: 2,717,226 (10.37 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# 10. Callbacks
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

In [66]:
# 11. Train
history = model.fit(
    X_train_vec, y_train,
    validation_split=0.1,
    epochs=15,
    batch_size=64,
    callbacks=callbacks
)

Epoch 1/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 292s 121ms/step - accuracy: 0.4097 - loss: 2.3645 - val_accuracy: 0.5634 - val_loss: 1.5919 - learning_rate: 0.0010
Epoch 2/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 298s 111ms/step - accuracy: 0.6180 - loss: 1.3758 - val_accuracy: 0.5815 - val_loss: 1.5229 - learning_rate: 0.0010
Epoch 3/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 258s 109ms/step - accuracy: 0.6865 - loss: 1.0862 - val_accuracy: 0.5784 - val_loss: 1.5857 - learning_rate: 0.0010
Epoch 4/15
2357/2358 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.7393 - loss: 0.8692
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 253s 107ms/step - accuracy: 0.7393 - loss: 0.8692 - val_accuracy: 0.5708 - val_loss: 1.6757 - learning_rate: 0.0010
Epoch 5/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 260s 106ms/step - accuracy: 0.8052 - loss: 0.6341 - val_accuracy: 0.5676 - val_loss: 1.9660 - learning_rate: 5.0000e-04


In [67]:
# 12. Evaluate
print("\n Final Test Results:")
model.evaluate(X_test_vec, y_test)



📊 Final Test Results:
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.5826 - loss: 1.5250


[1.5283540487289429, 0.5824941396713257]

In [68]:
def predict_category(texts, model, vectorizer, label_encoder):
    cleaned = [clean_text(t) for t in texts]
    vec = vectorizer(tf.constant(cleaned))
    preds = model.predict(vec)
    predicted_indices = np.argmax(preds, axis=1)
    return label_encoder.inverse_transform(predicted_indices)


sample_headlines = [
    "NASA discovers new planet that could support life",
    "Trump speaks at election rally in Florida",
    "New iPhone released with improved camera",
    "Tips for saving money on a tight budget",
    "Why yoga is the ultimate stress reliever"
]

predicted_labels = predict_category(sample_headlines, model, vectorizer, label_enc)

for headline, label in zip(sample_headlines, predicted_labels):
    print(f" Headline: {headline}\n ----> Predicted Category: {label}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
 Headline: NASA discovers new planet that could support life
 ----> Predicted Category: SCIENCE

 Headline: Trump speaks at election rally in Florida
 ----> Predicted Category: POLITICS

 Headline: New iPhone released with improved camera
 ----> Predicted Category: TECH

 Headline: Tips for saving money on a tight budget
 ----> Predicted Category: HOME & LIVING

 Headline: Why yoga is the ultimate stress reliever
 ----> Predicted Category: WELLNESS



In [ ]:
history = model.fit(
    X_train_vec, y_train,
    validation_split=0.1,
    epochs=15,
    batch_size=64
)

print("\n Final Test Results:")
model.evaluate(X_test_vec, y_test)

Epoch 1/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 251s 106ms/step - accuracy: 0.7097 - loss: 0.9981 - val_accuracy: 0.5831 - val_loss: 1.5966
Epoch 2/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 261s 106ms/step - accuracy: 0.7643 - loss: 0.7828 - val_accuracy: 0.5791 - val_loss: 1.7558
Epoch 3/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 249s 106ms/step - accuracy: 0.8104 - loss: 0.6269 - val_accuracy: 0.5685 - val_loss: 1.9977
Epoch 4/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 259s 105ms/step - accuracy: 0.8458 - loss: 0.5061 - val_accuracy: 0.5640 - val_loss: 2.2480
Epoch 5/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 257s 109ms/step - accuracy: 0.8733 - loss: 0.4118 - val_accuracy: 0.5593 - val_loss: 2.5084
Epoch 6/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 250s 106ms/step - accuracy: 0.8950 - loss: 0.3407 - val_accuracy: 0.5507 - val_loss: 2.7101
Epoch 7/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 267s 108ms/step - accuracy: 0.9106 - loss: 0.2855 - val_accuracy: 0.5450 - val_loss: 3.0483
Epoch 8/15
2358/2358 ━━━━━━━━━━━━━━━━━━━━ 254s 108ms/step - ac